In [29]:
import numpy as np
import matplotlib.pyplot as plt
from itertools import product as product
from scipy.linalg import expm
%matplotlib notebook

In [30]:
N = 100
I_0 = 5
Rsi = 2/N
Rir = 1/N

states = {(x, y) : ind for ind, (x, y) in enumerate([(x, y) for x in range(N + 1) for y in range(N + 1) if x + y <= N], 0)}
biStates = {ind : state for state, ind in states.items()}

In [31]:
Q = np.zeros((len(states), len(states)))

for (x, y), ind in states.items():
    if y > 0:
        transitionTo = states[(x, y - 1)]
        Q[ind, transitionTo] = Rir * y
    if x > 0:
        transitionTo = states[(x - 1, y + 1)]
        Q[ind, transitionTo] = Rsi * y * x
    Q[ind, ind] = - np.sum(Q[ind, :])

In [32]:
with np.printoptions(precision=2):
    print(Q)

[[-0.    0.    0.   ...  0.    0.    0.  ]
 [ 0.01 -0.01  0.   ...  0.    0.    0.  ]
 [ 0.    0.02 -0.02 ...  0.    0.    0.  ]
 ...
 [ 0.    0.    0.   ... -0.    0.    0.  ]
 [ 0.    0.    0.   ...  0.01 -1.99  0.  ]
 [ 0.    0.    0.   ...  0.    0.   -0.  ]]


In [33]:
Ps = [expm(Q * t) for t in [5, 10, 20, 50]]

In [34]:
for P in Ps:
    assert np.allclose(np.sum(P, axis=1), np.ones((P.shape[0],)))

In [35]:
# estimate expected value of I_t
Is = []

for P in Ps:
    assert np.allclose(np.sum(P, axis=1), np.ones((P.shape[0],)))
    start = states[(N - I_0, I_0)]
    Is.append(sum(biStates[col][1] * P[start, col] for col in range(P.shape[1])))

In [36]:
Is

[96.48317550170825, 91.90590030702678, 83.1599086606544, 61.60637556604254]